In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
import time 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.applications.densenet import DenseNet121, preprocess_input

In [7]:
dn_model = tf.keras.models.load_model('../models/mobilenetv2_model//')
class_names = {
    0: 'anger',
    1: 'contempt',
    2: 'sad',
    3: 'fear',
    4: 'disgust',
    5: 'happy',
    6: 'surprise'
}

In [8]:
#preprocessing image 
def preprocess_frame(frame, target_height=224, target_width=224):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    frame = cv2.resize(frame, (target_width, target_height))
    frame = frame.astype(np.float32) / 255.0
    return frame

In [12]:
# open webcam
cap = cv2.VideoCapture(0)

# take an image every 5 seconds 
start_time = time.time()
prediction_interval = 5

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture a frame. Exiting...")
        break

    # preprocess of frame 
    preprocessed_frame = preprocess_frame(frame)
    preprocessed_frame_batch = np.expand_dims(preprocessed_frame, axis=0)

    # making prediction 
    predictions = dn_model.predict(preprocessed_frame_batch)
    predicted_class_index = np.argmax(predictions[0])

    # labeling the class 
    predicted_class_label = class_names[predicted_class_index]

    # getting the %
    percentages = predictions[0] * 100

    # sorting classes with label 
    sorted_indices = np.argsort(percentages)[::-1]
    label_text = ""
    for index in sorted_indices:
        class_name = class_names[index]
        percentage = percentages[index]
        label_text += f"{class_name}: {percentage:.2f}%\n"
    cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
    cv2.imshow('Webcam', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 0s 89ms/step


In [13]:
cap.release()
cv2.destroyAllWindows()